# Assignment 01 - Hash Tables and Pitfalls in Data Mining     
## CSCI E-108

The goal of data mining is to find important relationships in large complex datasets. These datasets typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls which lead to incorrect inferences.   

In this assignment you will gain a bit of experience with two important concepts in data mining using a small scale example. These methods are essential as the scale increases:  
1. **False Discovery Rate Control:** The goal of data mining is to find important relationships in large complex datasets. These datasets typically contain a large number of variables. The **high-dimensional** nature of the data leads to a commonly encountered pitfall of incorrect inference, **false discovery**. A related problem is when a large-scale analysis is terminated when a desired relationship is 'found'. This practice of **p-value mining** often leads to unwarranted inferences. In the exercises in this notebook you will apply **false discovery rate (FDR) control** methods to address these problems.   
2. **Hashing and Key-Value Pairs:** Large scale data is typically managed using key-value (KV) pairs in hash tables. The exercises in this assignment give you some experience working with KV pair data management in a hash table.  

In order to keep the scope of this assignment manageable, you will use limited versions of KV pair management on a hash table. Specifically, you will use a simple Python list implement these concepts rather than dedicated large scale analytic platforms. 

To get started, execute the code in the cell below to load the required packages. 

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from scipy.stats import ttest_ind, f_oneway
from itertools import product, combinations
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
import math
import time
import sys

pd.set_option("display.max_rows", None, "display.max_columns", None)

## Synthetic Data for Multiple Hypothesis Testing

Testing multiple hypothesis in high-dimensional data is problematic. Exhaustively testing all pairwise relationships between variables in a data set is a commonly used, but generally misleading, form of **multiple comparisons**. The chance of finding false significance, using such a **data dredging** approach, can be surprisingly high. 

In the following exercises you will perform multiple comparisons on **20 identically distributed independent (iid)** variables. Ideally, such tests should not find significant relationships, but the actual result is quite different. Here we create a synthetic data set with 20 iid Normal variables of 1000 samples each. Execute the code in the cell below to compute this data by random sampling and display the mean and variance of each variable and the correlation between them.  

In [ ]:
ncolumns = 20
nr.seed(22349)
normal_vars = nr.normal(size=(1000,ncolumns))
print('The means of the variables are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the variables are\n', np.var(normal_vars, axis = 0))

_, ax = plt.subplots(figsize=(10,8))
sns.heatmap(np.corrcoef(np.transpose(normal_vars)), ax=ax);
ax.set_title('Correlations between 20 iid normal random variables');

Notice that means are close to 0.0 and variances are close to 1.0 respectively.    



## Hash Functions and Hash Tables  

Management of massive datasets requires using **key-value pairs** in a **hash table**. The keys are hashed and the hashes used as indices to reference buckets in the hash table. The buckets hold a tuple of the original key (unhashed) and the values.   
 
The question is, how can we create a good hash function for the the keys? Ideally, the hash function should produce keys with these closely related properties:   
1. The hash values should be unique given the keys to avoid hash collisions.
2. The hash values should be uniformly distributed given the keys. 

### Universal hash functions
**[Universal hash functions](https://en.wikipedia.org/wiki/Universal_hashing)** are fairly simple to implement and yet have generally good properties. Consider hashing a set of integers, $U = [i_1, i_2, i_3,..., u_u]$. We want to create a family of hash functions, $H(U) \rightarrow m$, to map $U$ to $m$ unique hash values. Ideally, we want the the probability of a hash value to be uniform, $p \big(h(i) \big) \le \frac{1}{m}$.    

For the integer keys, we will work work with a universal hash function in the form:     

$$h(i) = \big( (a\ i + b)\ mod\ p \big)\ mod\ m$$
where   
- a is a random multiplier, $0 \lt a \lt p -1$ and $a \ne 0$.
- b is a random additive factor, .   
- p is a (usually prime) number, $p \ge |U|$, $|U| = $ size for the set to be hashed.
- m is the size of the hash table.

In this case, we will be working with a vector of keys of length 2, $K=[k_1,k_2]$. The universal hash of an $n$ length vector of values can easily be computed:    

$$h(K) = \Big( \sum^n_{i=1} h_i (k_i) \Big)\ mod\ m$$
Where, each of the $h_1, h_2, ..., h_n$ are independent universal hash functions.   

> **Exercise 1-01:** In this notebook you will perform pairwise hypothesis tests on sets of vector-valued variables. There are $\frac{n (n-1)}{2}$ pairwise combinations. There are two keys, $[i,j]$, for this hash table.
> 
> We could simply manage the results set in a Pandas data frame and do not need a sophisticated and scaleable hash table. To fulfill one of the goals of these exercises, we will use a hash table with vector-valued keys.     
>
> In this exercise you will test a universal hash function using vector-valued keys. To complete the code and test this hash, do the following:    
> 1. Complete `hash_function` in the code cell below to implement the integer key universal hash function discussed the foregoing section. The function should return an integer hash in the range $[1,1024]$ given an integer key value. 
> 2. Complete the code that iterates over all combinations of the keys (indexes) of the 20 variables and displays the keys and hash values.
>    - Use the itertools package [combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) function to find all combinations of two keys in `range(ncolumns)`.
>    - For each combination of keys, $i, j$, find the hash value using the `multivariate_hash` function provided.
>    - Append the keys and hash value to `hash_list`.
> Execute your code and examine the results. 

In [ ]:
modulo=1024
a_s=[10147, 34519]
b_s=[2341, 7575]
p_s=[101273, 106949]   

def hash_function(key, a, b, p, modulo):
    """
    Function to compute hash for hash of x 

    Args: 
        x - integer value to be hashed 
        a - random multiplier value
        b - random addition value 
        p - random modulo value
        modulo - an integer = length of the hash table 

    Returns: a integer hash value     
    """
    ## Complete the code in this function 
   
    

def multivariate_hash(x, a, b, p, modulo): 
    """
    Function to compute hash for hash a vector x 

    Args: 
        x - vector or list of integer values 
        a - random vector of multiplier values of len(x)
        b - random vector of addition values of len(x) 
        p - random vector of modulo values of len(x) 
        modulo - an integer = length of the hash table 

    Returns: a integer hash value     
    """
    h = []
    for i, key in enumerate(x):
        h.append(hash_function(key, a[i], b[i], p[i], modulo))
    return sum(h) % modulo


hash_list = []
## Put your code below




    ## Print a sample of the hash values 
    if i <= 3:             
        print('i = ' + str(i) + '  j = ' + str(j) + '   hash = ' + str(hash_value))
#    return(hash_list)        
#hash_list = print_hashes(a=a_s, b=b_s, p=p_s, modulo=modulo)            

> 3. The next question is how good is this hash function. We can run a simple test by plotting the hash values computed. Execute the code in the cell below to display a histogram and a cumulative distribution plot of the hash values. 

In [ ]:
def plot_hash(hashes, bins=1024, modulo=1024):
    fig, ax = plt.subplots(figsize=(20, 6))
    counts, bins, _ = ax.hist(hashes, bins=bins);
    ax.set_ylabel('Count of keys');
    ax.set_xlabel('Hash value');
    ax.set_title('Count of keys vs. hash values');
    ax.set_xlim(0.0, modulo)
    plt.show();

    fig, ax = plt.subplots(figsize=(6,6))
    ax.plot(range(len(counts)), np.cumsum(counts))
    ax.plot([0.0,len(counts)], [0.0,len(hashes)], color='red');
    ax.set_ylabel('Cumulative Sum');
    ax.set_xlabel('Hash index');
    ax.set_title('Cumulative sum of hash values');
    ax.set_xlim(0.0, modulo)
    plt.show();
    
plot_hash(hash_list)    

> Examine these results and answer the following questions in one or two sentence.
> 1. Are hash collisions evident?     
> 2. Based on the histogram and cumulative distribution plot does this hash function have nearly ideal behavior, and what deviations can you observe?
> 3. Hypothetically, we could reduce the number of hash collisions by using a longer hash table. For massive scale problems why is this approach not practical?  
> **End of exercise.**

> **Answers:**    
> 1.           
> 2.       
> 3.        

### Linear probing        

Given the inevitable hash collisions we need a method to resolve them. **[Linear probing](https://en.wikipedia.org/wiki/Linear_probing)** is an effective, efficient and widely used hash collision resolution algorithm. The basic ideas is simple. If a hash bucket is occupied the algorithm linearly scans the table until an open bucket is found. The keys in each bucket are then used for look-ups.   

> **Exercise 1-02:** You will now compute the two missing lines of code to implement a simple linear probing algorithm. The algorithm operates as follows:
> 1. The bucket of the hashed key value is tested to see if it is empty. **Complete the code required for this case to add the key-value tuple to the table.**
> 2. If the first bucket is used, the linear probing starts with a while statement. The loop continues until an empty bucket is found or no empty bucket is available.
> 3. The hash value is incremented by 1.
> 4. If the new hash value is greater than the table length the hash value is set to 0, the beginning of the table. If the flag showing the table has started from the beginning already, the function returns `None` since there are no open hash buckets. Otherwise, the flag is set to 1 and probing for an open bucket continues.
> 5. **Complete the code to add the key-value tuple to the table for the case when an open buck in found in the linear search.**
>    
> Execute your code and examine the results.  

In [ ]:
def instantiate_hash(length):  
    """
    function instantiates a hash table of lenght 'length'
    """
    return [None]*length
   

def linear_probing_insert(hash_value, kv_pair, hash_table):   
    """
    Function to insert key-value pairs into an existing hash table 

    Args
        hash_value - the hash index, int
        kv_pair - the k-v pair to inser4t as tuple, (key, value)
        hash_table - in the rofm of Python list    

    Returns: hash_table
    """
    if hash_table[hash_value] is None: 
        ## Complete the code below
        
         
    else: 
        hash_len = len(hash_table)
        flag = 0
        while hash_table[hash_value] is not None:
            hash_value += 1 # Increment the hash_value to prob of open bucket 
            if hash_value > hash_len - 1: # Don't want to run off end of table
                if flag == 1: # If we have been here before the table must be full  
                    print('No open buckets found')
                    return None 
                # wrap around to the beginning of the table to continue search 
                hash_value = 0   
                flag = 1
            if hash_table[hash_value] is None: 
                ## Complete the code below
                
                
                break
    return hash_table

hash_table = instantiate_hash(modulo)
for i,j in combinations(range(ncolumns), 2):
        hash_value = multivariate_hash([i,j], a_s, b_s, p_s, modulo)
        linear_probing_insert(hash_value, ((i,j), None), hash_table)

print('Sample of hash buckets')
print(hash_table[:50])
full_buckets = [0 if t is None else 1 for t in hash_table]
print(f"\nNumber of non-empty buckets: {sum(full_buckets)}")

> Next execute the code to display an histogram and cumulative distribution plot of the hash values once linear probing has been applied.    

In [ ]:
ls = []
for idx in range(len(hash_table)):
    if hash_table[idx] is not None: ls.append(idx) 

print(f"The number of buckets: {len(ls)}")
plot_hash(ls) 

> Examine these results and answer these questions:      
> 1. What evidence do you see that the hash collisions been resolved by the linear probing.
> 2. Has application of linear probing significantly affected the cumulative distribution function.         
> **End of exercise.**  

> **Answers:**    
> 1.         
> 2.          

## Adding Values to the Hash Buckets    

Now that hashing with linear probing has been implemented and a hash table instantiated with the keys, it is time to add the values to the table. At this point, the hash table only contains the keys, but no values. Specifically, you will add a tuple of the t-statistic and p-value for each of the k-v pairs already in the table. Recalling that the key has two values, you need to iterate over all the combinations of the random variable vectors (columns) to compute the pairwise statistics. The tuple of the keys and the tuple of the t-statistic and p-value update the contents of the hash bucket. The table now has complete k-v pairs for every combination of the variables being tested.         

There are two parts to the required processing. First, a function to update a bucket in the hash table given the linear probing is required. Second, you will need to compute the t-statistic and p-value and update the value of the appropriate bucket using the update function.

The `linear_prob_update` function follows a a pattern similar to the `linear_probing_insert` function you have already worked with. The flow of the function is as follows:
1. The bucket for the hashed key value is tested to see if the key is in that bucket. If so, the update is placed in the bucket. 
2. If the first bucket does not contain the required key, the linear probing starts with a while statement. The loop continues until the key is found, or it is determined that the key is not present in the table.
3. The hash value is incremented by 1.
4. If the new hash value is greater than the table length the hash value is set to 0, the beginning of the table. If the flag showing the table has started from the beginning already, the function returns `None` since there are no open hash buckets. Otherwise, the flag is set to 1 and probing for key continues.
5. Once the probing finds the required key the update is made to the hash table bucket.  

> **Exercise 1-03:** 
> The next task is to compute the required statistics and update the bucket in the has table with a tuple of these values. 
> 1. Iterate over all **combinations** of column indices for the `normal_vars` array. These indices are also the keys for the hash table.
> 2. Use the [scipy.stats.ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) function, with default arguments for the hyperparameters, to compute the t-statistic and p-value.
> 3. Compute the hashed key value using the `multivariate_hash` function.
> 4. Use the `linear_prob_update` function to update the hash table.
>    
> Execute your code.     

In [ ]:
def linear_prob_update(hash_value, key, value, hash_table):    
    """
    Function to update value of the key-value pairs into an existing hash table 

    Args
        hash_value - the hash index, int
        key - the key tuple of the bucket to update
        value - value tuple to add to hash tabel
        hash_table - hash table to update

    Returns: hash_table
    """
    if hash_table[hash_value][0] == key: 
        # First the simple case, finding the bucket on the first try
        hash_table[hash_value] = (key, value) 
        return hash_table
    else: ## need to prob for correct bucket  
        hash_len = len(hash_table)
        flag = 0
        while hash_table[hash_value][0] != key: 
            hash_value += 1 # Increment the hash_value to prob of open bucket 
            if hash_value > hash_len - 1: # Don't want to run off end of table
                if flag == 1: # If we have been here before the table must be full  
                    print('Key not found!')
                    return None 
                # wrap around to the beginning of the table to continue search 
                hash_value = 0   
                flag = 1
            if hash_table[hash_value][0] == key: 
                # We found the bucket! 
                hash_table[hash_value] = (key, value)   
                return hash_table

## Put your code below     




hash_table[:50]

> You should be able to see the key-value pairs in some buckets of the hash table.   
> **End of exercise.**

## Querying the Hash Table

With the t-statistics and p-values computed, the next question to ask is how many of these 
How many of these t-tests will show **significance** at the 0.05 cut-off level? With 20 variables there are 190 unique pairwise combinations. We expect to find a number of falsely significant test results. This task can be accomplished by querying the table for all the key tuple pairs and applying a significance test to the p-value. The number of positive results is then summed.   

THe `linear_prob_query` function follows a a pattern similar to the `linear_probing_update` function you have already worked with. The flow of the function is as follows:
1. The bucket of the hashed key value is tested to see if the key is in that bucket. If so, the value in the bucket is returned. 
2. If the first bucket does not contain the required key, the linear probing starts with a while statement. The loop continues until the key is found, or it is determined that the key is not present in the table.
3. The hash value is incremented by 1.
4. If the new hash value is greater than the table length the hash value is set to 0, the beginning of the table. If the flag showing the table has started from the beginning already, the function returns `None` since there are no open hash buckets. Otherwise, the flag is set to 1 and probing for key continues.
5. Once the probing finds the required key the value in the bucket is returned.  

> **Exercise 1-04:** You will now complete the code in the `test_significance` function below by these steps.
> 1. The `linear_prob_query` function is called with the hashed value, the key and the hash table. The t-statistic and p-value are returned. 
> 2. A logical test function, `standard_hypthesis_test`, called to determine if the p-value is significant given cutoff value. If so, the `significant` variable is set to `True`, else it is set to `False`. **Complete this function.**
> 3. The function will then update the value in the hash bucket and return the hash table.  

In [ ]:
def linear_prob_query(hash_value, key, hash_table):    
    """
    Function to query the value of key-value pairs stored in a hash table 

    Args
        hash_value - the hash index, int
        key - the key tuple of the bucket to update
        hash_table - hash table to update

    Returns: The value of the key-value pair in the hash bucket
    """
    if hash_table[hash_value][0] == key: 
        # First the simple case, finding the bucket on the first try
        return hash_table[hash_value][1]
    else: ## need to prob for correct bucket  
        hash_len = len(hash_table)
        flag = 0
        while hash_table[hash_value][0] != key: 
            hash_value += 1 # Increment the hash_value to prob of open bucket 
            if hash_value > hash_len - 1: # Don't want to run off end of table
                if flag == 1: # If we have been here before the table must be full  
                    print('Key not found!')
                    return None 
                # wrap around to the beginning of the table to continue search 
                hash_value = 0   
                flag = 1
            if hash_table[hash_value][0] == key: 
                # We found the bucket! 
                return hash_table[hash_value][1]

def test_significance(significance_level, test_function, hash_value, key, hash_table, q=1):    
    """
    Function to apply a significane test function to an hypothesis test  

    Dependencies:  
        linear_prob_query
        linear_prob_update

    Args
        significance_level - the cutoff value for the hypothsis test 
        test_function - a Python function used to determine if a test result is significant 
                       Arguments: 
                           significance_level
                           p-value 
                           q - parameter used for some test algorithms   
                       Regturns: logical, True or False
        hash_value - the hash index, int
        key - the key tuple of the bucket to update
        hash_table - hash table to update
        q - an optional parameter used for some significant test algorithms 
                       

    Returns: hash_table
    """
    ## Your code goes here
    


    
    return hash_table 

def call_function(func, **kwargs):
    ## Function to call the significance evaluation function 
    return func(**kwargs)


def standard_hypothesis_test(p_value, significance_level):
    ## Put your code below 
    if p_value <= significance_level: return True
    else: return False 


significance_level = 0.05 
for i,j in combinations(range(ncolumns), 2):
    hash_value = multivariate_hash([i,j], a_s, b_s, p_s, modulo)
    hash_table = test_significance(significance_level, standard_hypothesis_test, hash_value, (i,j), hash_table)
    
hash_table[:50]

> 4. The code in the cell below sums the number of pairs found to be significant. Execute this code.  

In [ ]:
def count_significant(hash_table):
    count = 0
    for i,j in combinations(range(ncolumns), 2):
        hash_value = multivariate_hash([i,j], a_s, b_s, p_s, modulo)
        count += linear_prob_query(hash_value, (i,j), hash_table)[2]
    return count    

print(f"Number of false discoveries: {count_significant(hash_table)}")

> Based on the foregoing result, answer these questions:
> 1. With a cutoff value of 0.05 what is the expected number of false discoveries?
> 2. Is the number of false discoveries consistent with the expected number?
>    
> **End of exercise.**

> **Answers:**
> 1.      
> 2.            

### Bonferroni correction  

Several adjustments to the multiple comparisons problem have been proposed. Dunn published a method know as the **Bonferroni correction** in 1961. The Bonferroni correction is a widely used method to reduce the false positive rate of hypothesis tests. The adjustment is simple:
$$\alpha_b = \frac{\alpha}{m} \\
with \\ 
m =\ number\ of\ groups$$

Can the Bonferroni correction help? Yes, by greatly increasing the confidence level required for a statistically significant result. The problem with the Bonferroni correction is the reduction in power as the  grows smaller. For big data problems with large numbers of groups, this issue can be especially serious. In fact, the Bonferroni correction can often lead to significant numbers of missed discoveries, or Type II errors. 

> **Exercise 1-05:** You will now apply the Bonferroni correction to determining the significance of the p-values of the pair-wise t-tests. Complete the code to compute `q`, the cutoff value for the Bonferroni significance test. You can compute the number of tests using the [math.comb](https://docs.python.org/3/library/math.html#math.comb) function.  
> 
> Execute your code and examine the results

In [ ]:
significance_level = 0.05 

## Put your code below   



for i,j in combinations(range(ncolumns), 2):
    hash_value = multivariate_hash([i,j], a_s, b_s, p_s, modulo)
    hash_table = test_significance(q, standard_hypothesis_test, hash_value, (i,j), hash_table)
    
print(f"With Bonferonni cutoff value: {q}")
print(f"Number of false discoveries: {count_significant(hash_table)}")   

> Given the results, answer these questions:
> 1. Does using the  Bonferroni correction reduce false significance tests, or even eliminate them?
> 2. Given the Bonferroni correction and this difference in means, would this result be considered significant or would it be a type II error, and why?    
> 3. What does this result tell you about the downside of the Bonferroni correction as a FDR control method, in terms of finding significant results? 
> **End of exercise.**    

> **Answers:**
> 1.       
> 2.        
> 3.          

## Another Dataset

To continue with the example, execute the code in the cell below to load the data and print the dimensionality of the data frame. 

In [ ]:
gene_data = pd.read_csv('../data/ColonDiseaseGeneData-Cleaned.csv')
print(gene_data.shape)

There are data from 97 patients (rows) with 10,497 gene responses (features in columns). The specific problem we will explore is to determine which genes lead to expression of a certain disease. In this example, there are gene expression data for 97 patients. Some of these patients have ulcerative colitis, a condition believed not to be inheritable, and others have Crohn's disease, which is believed to be genetically inherited. One approach to this problem is to perform hypothesis tests on the expression each of the genes between patients with the two conditions. Since there are over 10,000 genes there is considerable chance for false discovery. Therefore, careful application of FDR control is required!

Execute the code in the cell below to view the first 5 columns of the data frame, which includes the expression of the first 4 genes.  

In [ ]:
 print(gene_data.iloc[:,:5])      

We need to perform two transformations of the gene response data before we can perform any analysis.    
1. Create two data frames, one for each patient condition.       
2. Take the transpose of the data frames so that the gene response vectors are in the rows.

Execute this code and examine the response.  

In [ ]:
def map_gene(gene_data):  
    ## Function separates the columns by disease type  
    mask = gene_data.loc[:,'Disease State'] == 'Ulcerative Colitis (UC)'
    ulcerative = gene_data[mask]
    crohns = gene_data[np.logical_not(mask)]
    ## Return the transpose of the two data frame 
    return ulcerative.iloc[:,1:].transpose(), crohns.iloc[:,1:].transpose()
    
ulcerative, crohns = map_gene(gene_data)    
print(f"\nUlcerative Colitis data frame of dimension: {ulcerative.shape}")
print(ulcerative.iloc[:10,:10])
print(f"\nCorhn's data frame of dimension: {crohns.shape}")
print(crohns.iloc[:10,:10])

You can see that there are 10,496 gene expression values for each of the 24 Corhn's patients and 73 colitis patients.  

### Simple significance test on gene response data

You will now apply both a standard hypothesis test and the static Bonferonni corrected hypothesis test to the gene expressions for the two patient populations. As a first step, execute the code in the cell below to compute a key-value pair table with the following content:    
- The keys are the gene identifiers
- The values are columns containing the t-statistic, the p-value, and a logical value of significance for the t-test on the gene.

In the interest of simplicity we are using a standard data frame as our k-v pair hash table. Data frames do in fact use a hash of the row and column indices to perform operations on the table. In this case, we will iterate though a key list to perform row-wise operations on the table.   

In [ ]:
def plot_p_values(results):
    """
    Function to plot the distribution of p-values
    """
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    sns.histplot(x='p_value', data=results, bins=100, ax=ax[0])
    ax[0].set_yscale('log')
    ax[0].set_title('Distribution of p_values')
    ax[0].set_xlabel('p-value')
    ax[0].set_ylabel('Log density')
    
    sns.histplot(x='p_value', data=results, bins=500, ax=ax[1])
    ax[1].set_yscale('log')
    ax[1].set_xlim(0, 0.1)
    ax[1].set_title('Distribution of p_values\n limited x-range')
    ax[1].set_xlabel('p-value')
    ax[1].set_ylabel('Log density')

def gene_t_test(corhns, ulcerative, keys):   
    '''
    Function to scan the tables of gene expression for the two patient populations and perform gene wise t-tests.   

    Args: 
        crohns - data frame of gene responses for Corhn's disease patients with rows keyed by gene identifier. 
        ulcerative - data frame of gene responses for ulerative cholitis patients with rows keyed by gene identifier.
        keys - a list or array of gene key values as strings   
    Returns: 
        A data frame with gene identifier as row key and with the following columns:  
        t_statistic 
        p_value
    '''
    t_test = pd.DataFrame(columns = ['t_statistic', 'p_value', 'significant'], index = keys)
    for key in keys: 
        t_statistic, p_value = ttest_ind(corhns.loc[key,:], ulcerative.loc[key,:])
        t_test.loc[key,:] = [t_statistic, p_value, None]

    return t_test    


# Get a list of the key values
key_list = crohns.index

t_test_results = gene_t_test(crohns, ulcerative, key_list)
plot_p_values(t_test_results)

From these plots one can see that there are a large number small of p-values. This situation is likely to result in high false discovery rate.   

> **Exercise 1-06:** You will now apply the standard hypothesis test method with a fixed significance level and no false discovery rate control to the gene expression dataset, as a basis of comparison. To do so, you must complete the code for the `test_gene_significance` function. The function applies cutoff threshold or `significance_level` to the p-value and returns a logical, $\{True,False \}$. When you have completed the code execute it.  

In [ ]:
def test_gene_significance(p_value, cutoff): 
    ## Put your code below 
   

    

significance_level = 0.05 
for key in key_list:
    t_test_results.loc[key,'significant'] = test_gene_significance(t_test_results.loc[key,'p_value'], significance_level)

print(f"Number of significant tests: {sum(t_test_results.loc[:,'significant'])}")

> **End of exercise.**

### Bonferonni significance test on gene response

> **Exercise 1-07** With the basic test completed, you will now attempt to control the false discovery rate with the Bonforonni method. Execute the code in the cell below with the static cutoff value or `significance_level` computed using the cutoff value and number of tests.    

In [ ]:
cutoff = 0.05

## Put your code below




for key in key_list:
    t_test_results.loc[key,'significant'] = test_gene_significance(t_test_results.loc[key,'p_value'], significance_level)

print(f"Bonferonni cutoff level for significance: {significance_level}")
print(f"Number of Bonferonni significant tests: {sum(t_test_results.loc[:,'significant'])}")

> Compare the results for the basic hypothesis test and Bonforonni correction and answer these questions:
> 1. Do you think it is likely the basic hypothesis test leads to a high false discovery rate and why?
> 2. Given the Bonferonni cutoff value do you think it is likely to lead to missed discoveries and why?
> 3. Do you think it would be easy to parallelize the algorithms used for the basic and Bonforonni corrected hypothesis tests and why?
>
> **End of exercise.** 

> **Answers:**
> 1.          
> 2.    
> 3.        

## Improved False Discovery Rate Control Methods 

We have seen the potential pitfalls of multiple hypothesis testing. Further, we have seen that a simple Bonforonni approach to [**false discovery rate (FDR) control**](https://en.wikipedia.org/wiki/False_discovery_rate) is overly conservative may not be effective. You will now apply two more sophisticated FDR control methods, Holm's method and the Benjamini-Hochberg FDR control algorithm. These methods control the FDR, while not being overly conservative on true positive results, like the Bonferronic correction.        

These FDR control use the observation that p-values of the multiple hypothesis tests can be ordered. A dynamic threshold is applied to the ordered p-values to improve results compared to static methods like Bonforonni that use one threshold for all p-values. The dynamic threshold allows for both reduced false discovery rate while reducing the probability missed true discoveries.    

### Holm's method

The first of these **Holm's method**. Holm's method operates on the ordered set of p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$. The threshold is dynamic with the order of the P-value. For the $ith$ p-value, $p(i)$ for $N$ pairwise tests is:  

$$p(i) \le Threshold(Holm's) = \frac{\alpha}{N - i + 1}$$

For example: for the 10th ordered p-value with 1,000 total tests (genes) and significance level of 0.05, the cutoff is:   

$$p(10) \le \frac{0.05}{1000 - 10 + 1} = 0.00005045$$

> **Exercise 01-08:** You will now apply the dynamic Holm's FDR control method to the gene data. Do so by the following steps:
> 1. The table sorted by p-value in ascending order.     
> 2. An updated key list is created from the ordered table.
> 3. **Complete the code** to compute the Holm's significance levels by appending the cutoff values to the `significance_levels` list. Make sure you account for the zero-based indexing used in Python when computing the denominator.   
> 4. The significance tests are applied to the ordered p-values in the table.
> 5. The number of significant tests is computed and displayed.     

In [ ]:
t_test_results.sort_values(by='p_value', ascending=True, inplace=True)
key_list = t_test_results.index

significance_level = 0.05 
significance_levels=[]

## Put your code below



## The significance levels are added to the data frame 
t_test_results['significance_levels'] = significance_levels


for i, key in enumerate(key_list):
    t_test_results.loc[key,'significant'] = test_gene_significance(t_test_results.loc[key,'p_value'], significance_levels[i])

print(f"Number of Holm significant tests: {sum(t_test_results.loc[:,'significant'])}")

> With the results from the Holm's corrected significance test computed, execute the code in the cell below to display a graph summarizing these results.   

In [ ]:
def plot_significance(results):
    results['test_number'] = range(len(results))
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.lineplot(x='test_number', y='significance_levels', data=results, ax=ax, color='black', linewidth=0.5)
    sns.scatterplot(x='test_number', y='p_value', hue='significant', data=results, s=3, ax=ax)
    ax.set_title('Significance of gene expression')
    ax.set_xlabel('Gene number')
    ax.set_ylabel('p-value')
    
plot_significance(t_test_results.iloc[:150,:].copy())    

> Compare the results of the Bonforonni correction to the Holm's method for the differences in gene expression between the two patient populations and answer these questions.
> 1. Compare the significant gene expressions found by the two methods.
> 2. Given the formulation of the Holm's method why are the cutoffs found to very dynamic for this specific situation?
>
> **End of exercise.** 

> **Answers:**
> 1.          
> 2.       

### Benjamini-Hochberg FDR Control 

The Benjamini-Hochberg FDR control algorithm is another way to control false discoveries. Stat with an ordered set of $n$ p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$ we define a false discovery rate, $q$:

$$FDR(D) \le q$$

The cutoff threshold for the ith p-value is then:
$$p_{(i)} \le Threshold(D_q) = \frac{q}{n} i$$

> **Exercise 01-09:** In this exercise you will apply the Benjamini-Hochberg FDR control algorithm for testing the significance of the gene expressions. The `BH_significance` function is quite similar to the Holm's method function you have already created. Given the large number of genes you must use a low false discovery rate, $q = 0.001$, or 1 out of 1,000. You will now proceed by the following steps:
> 1. **Complete the code** to compute the BH significance levels by appending the cutoff values to the `significance_levels` list. Make sure you account for the zero-based indexing used in Python when computing the denominator.   
> 2. The significance tests are applied to the ordered p-values in the table.
> 3. The number of significant tests is computed and displayed.
> 4. Display a plot summarizing the BH test results.
> 
> Execute the code.

In [ ]:
significance_level = 0.05 
q = 0.001
significance_levels=[]

## Put your code below  




## Add the BH significance levels to the data frame
t_test_results['significance_levels'] = significance_levels


for i, key in enumerate(key_list):
    t_test_results.loc[key,'significant'] = test_gene_significance(t_test_results.loc[key,'p_value'], significance_levels[i])

print(f"Number of Benjamini-Hochberg significant tests: {sum(t_test_results.loc[:,'significant'])}")
plot_significance(t_test_results.iloc[:150,:].copy())  

> Compare the plots of results of the Benjamini-Hochberg FDR control method to those of Holm's method and answer the following questions.
> 1. Why does the Benjamini-Hochberg FDR control method give a more dynamic or adaptive result for the case where $i << n$?
> 2. The BH cutoff values have been computed allowing an expected false discovery rate of $ q \times N = 0.001 \times 10000 = 10$. What would be the effect of using a lower value of $q$ and why might this change not be a desirable?

> **Answers:**
> 1.            
> 2.           

**Note:** In practice, one should investigate using several values of $q$ for the BH FDR control algorithm to determine the effect of this parameter on the results. We will skip this step here in the interest of time.  

##### Copyright 2021, 2022, 2023, 2024, 2025 Stephen F. Elston. All rights reserved. 